## Exploring Langchain - Part I

This practice takes me through the basics of langchain framework, deepening my understanding of how it acts as a sandwich between user created applications and LLMs.

Topics Covered: OpenAI LLM, Simple Sequential Chain, Sequential Chain, Agents and Memory.

**NOTE: This is just a practice notebook that acts as a precursor to the main project**

In [1]:
import pandas as pd

In [2]:
openapi_key = str(pd.read_csv("key.txt"))
key = ""

In [3]:
from langchain_openai import OpenAI


In [5]:
#temperature refers to model creativity with 0 being safe and 1 being most creative. Being creative may lead to factually incorrect results.

llm = OpenAI(temperature = 0.7, api_key=key) 

In [45]:
name = llm("I want to open a restaurant for Italian food. Suggest a fancy name for it.")
name

'\n"Bella Cucina" '

In [16]:
from langchain.prompts import PromptTemplate

prompt_template_name = PromptTemplate(input_variables = ['cuisine'],
                                     template = "I want to open a restaurant for {cuisine} food. Suggest a fancy name for it.")

prompt_template_name.format(cuisine = "Indian")

'I want to open a restaurant for Indian food. Suggest a fancy name for it.'

In [17]:
from langchain.chains import LLMChain

chain = LLMChain(llm = llm, prompt=prompt_template_name)
chain("Indian")

C:\Users\DELL\AppData\Local\Temp\ipykernel_22856\3975690982.py:3: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm = llm, prompt=prompt_template_name)
C:\Users\DELL\AppData\Local\Temp\ipykernel_22856\3975690982.py:4: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain("Indian")


{'cuisine': 'Indian', 'text': '\n\n"Spice Palace"'}

### Simple Sequential Chain

- One input and one output
- Intermediate steps where output of the first step becomes input of the second step.




In [55]:
llm = OpenAI(temperature=0.6, api_key=key)

In [14]:
# First chain that asks for restaurant name
prompt_template_name = PromptTemplate(
    input_variables=['cuisine'],
    template="I want to open a restaurant for {cuisine} food. Suggest a fancy name for this"
)

name_chain = LLMChain(llm = llm, prompt=prompt_template_name)

# Second chain that uses the restaurant name to generate food items
prompt_template_name = PromptTemplate(
    input_variables=['resturant_name'],
    template="I want menu items for {restaurant_name}. Return me as a csv list"
)

food_items_chain = LLMChain(llm = llm, prompt=prompt_template_name)

NameError: name 'PromptTemplate' is not defined

In [60]:
from langchain.chains import SimpleSequentialChain

chain = SimpleSequentialChain(chains = [name_chain, food_items_chain])
response = chain.run("Indian")
print(response)

C:\Users\DELL\AppData\Local\Temp\ipykernel_23856\1510711124.py:4: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chain.run("Indian")




1. Samosa
2. Chicken Tikka Masala
3. Lamb Vindaloo
4. Vegetable Biryani
5. Tandoori Chicken
6. Garlic Naan
7. Chana Masala
8. Palak Paneer
9. Butter Chicken
10. Mango Lassi


### Sequential Chain
If you want to print intermediate steps as well like the restaurant name

- multiple inputs and multiple outputs

In [13]:
# First chain that asks for restaurant name
llm = OpenAI(temperature=0.6, api_key=key)

prompt_template_name = PromptTemplate(
    input_variables=['cuisine'],
    template="I want to open a restaurant for {cuisine} food. Suggest a fancy name for this"
)

name_chain = LLMChain(llm = llm, prompt=prompt_template_name, output_key="restaurant_name")

# Second chain that uses the restaurant name to generate food items
llm = OpenAI(temperature=0.6, api_key=key)


prompt_template_items = PromptTemplate(
    input_variables=['restaurant_name'],
    template="I want menu items for {restaurant_name}. Return me as a csv list"
)

food_items_chain = LLMChain(llm = llm, prompt=prompt_template_items, output_key="menu_items")

C:\Users\DELL\AppData\Local\Temp\ipykernel_22856\3007395112.py:2: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(temperature=0.6, api_key=key)


NameError: name 'PromptTemplate' is not defined

In [77]:
from langchain.chains import SequentialChain

SequentialChain(
    chains = [name_chain,food_items_chain],
    input_variables = ['cuisine'],
    output_variables = ['restaurant_name', 'menu_items'])

print(chain.run('Indian'))




1. Samosa Chaat, $8.99
2. Chicken Tikka Masala, $14.99
3. Lamb Biryani, $16.99
4. Vegetable Korma, $12.99
5. Garlic Naan, $3.99
6. Mango Lassi, $4.99
7. Tandoori Chicken, $15.99
8. Palak Paneer, $13.99
9. Malai Kofta, $14.99
10. Gulab Jamun, $5.99


## Agents

In [11]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.llms import OpenAI

tools = load_tools(['wikipedia','llm-math'],llm = llm)

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
verbose = True)
#Thought and reaction - mimics that action

agent.run("When was Elon born and how old is he?")



> Entering new AgentExecutor chain...
 I need to use Wikipedia to find information about Elon Musk
Action: wikipedia
Action Input: "Elon Musk"
Observation: Page: Elon Musk
Summary: Elon Reeve Musk  (; born June 28, 1971) is a businessman known for his key roles in the space company SpaceX and the automotive company Tesla, Inc. His other involvements include ownership of X Corp., the company that operates the social media platform X (formerly Twitter), and his role in the founding of the Boring Company, xAI, Neuralink, and OpenAI. In November 2024, United States president-elect Donald Trump appointed Musk as the co-chair of the proposed Department of Government Efficiency (DOGE) in the second Trump administration. Musk is the wealthiest individual in the world; as of November 2024, Forbes estimates his net worth to be US$304 billion.
Musk was born in Pretoria, South Africa, and briefly attended the University of Pretoria before immigrating to Canada at the age of 18, acquiring citizen

'Elon Musk was born on June 28, 1971 and is currently 53 years old.'

## Memory


In [40]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

chain = LLMChain(llm = llm , prompt = prompt_template_name, memory=memory)
name = chain.run("Indian")
print(name)



"Maharaja's Palace" 


In [23]:
name = chain.run("Mexican")

In [24]:
print(chain.memory.buffer)

Human: Indian
AI: 

"Maharaja's Palace" 
Human: Mexican
AI: 

"El Jardín de Sabor" (The Garden of Flavor)


In [45]:
#Using window memory to track only the last n conversations

from langchain.chains import ConversationChain 
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=3)

convo = ConversationChain(llm = OpenAI(temperature=0.7, api_key = key),
                        memory = memory)
print(convo.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


In [35]:
convo.run("Who on the first cricket world cup?")

" The first cricket world cup was held in 1975 and was won by the West Indies team, captained by Clive Lloyd. The runners-up were Australia, led by Ian Chappell. The tournament was held in England and featured eight teams, with the final being played at Lord's Cricket Ground."

In [36]:
convo.run("What is 4+7?")

'  4+7 is equal to 11.'

In [37]:
convo.run("Who on the first football world cup?")

' The first football world cup was held in 1930 and was won by Uruguay, who beat Argentina 4-2 in the final. The tournament was held in Uruguay and featured 13 teams, with the final being played at the Estadio Centenario.'

In [38]:
convo.run("Who on the first hockey world cup?")

' The first hockey world cup was held in 1971 and was won by Pakistan, who beat Spain 1-0 in the final. The tournament was held in Barcelona, Spain and featured 10 teams, with the final being played at the Estadi del FC Barcelona.'

In [39]:
print(convo.memory.buffer)

Human: What is 4+7?
AI:   4+7 is equal to 11.
Human: Who on the first football world cup?
AI:  The first football world cup was held in 1930 and was won by Uruguay, who beat Argentina 4-2 in the final. The tournament was held in Uruguay and featured 13 teams, with the final being played at the Estadio Centenario.
Human: Who on the first hockey world cup?
AI:  The first hockey world cup was held in 1971 and was won by Pakistan, who beat Spain 1-0 in the final. The tournament was held in Barcelona, Spain and featured 10 teams, with the final being played at the Estadi del FC Barcelona.
